<a href="https://colab.research.google.com/github/merazlab/cuda/blob/main/gpu_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Checks nvidia-cuda compiler version

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Install NVCC Plugin for Jupyter notebook

https://github.com/andreinechaev/nvcc4jupyter

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-oedvsl3s
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-oedvsl3s
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=b833f9053f43c752cc101ca979867dfdb44fd2bd75af5350e67eb95076c11ab2
  Stored in directory: /tmp/pip-ephem-wheel-cache-il8417e1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [10]:
%%cu
#include<stdio.h>
#define BS 8
#define N 10

void print(int *A,int n){
    for(int i=0; i<n; i++)
        printf("%d ",A[i]);
}

__global__ void increment(int *A,int n){
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if(i < n) A[i]++;
}

int main(void){
    int threadsPerBlock, blocksPerGrid, n, *A, *dA;
    n=N; threadsPerBlock=BS;
    blocksPerGrid = (n+BS-1)/BS;
    A=(int*) malloc(n*sizeof(int));
    for(int i=0; i<n; i++) A[i]=i*10;
    //0 10 20 30 40 50 60 70 80 90 
    cudaMalloc((void**)&dA,n*sizeof(int));
    cudaMemcpy(dA,A,n*sizeof(int),cudaMemcpyHostToDevice);
    increment<<<blocksPerGrid,threadsPerBlock>>>(dA,n);
    cudaMemcpy(A,dA,n*sizeof(int),cudaMemcpyDeviceToHost);
    print(A, n);
    //1 11 21 31 41 51 61 71 81 91 
    cudaFree(dA);
    free(A);
    return 0;
}

1 11 21 31 41 51 61 71 81 91 


## Save the cuda file in using notebook and Run using notebook

Save the program with a filename, using writefile command

In [16]:
%%writefile increment.cu
#include<stdio.h>
#define BS 8
#define N 10

void print(int *A,int n){
    for(int i=0; i<n; i++)
        printf("%d ",A[i]);
}

__global__ void increment(int *A,int n){
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if(i < n) A[i]++;
}

int main(void){
    int threadsPerBlock, blocksPerGrid, n, *A, *dA;
    n=N; threadsPerBlock=BS;
    blocksPerGrid = (n+BS-1)/BS;
    A=(int*) malloc(n*sizeof(int));
    for(int i=0; i<n; i++) A[i]=i*10;
    //0 10 20 30 40 50 60 70 80 90 
    cudaMalloc((void**)&dA,n*sizeof(int));
    cudaMemcpy(dA,A,n*sizeof(int),cudaMemcpyHostToDevice);
    increment<<<blocksPerGrid,threadsPerBlock>>>(dA,n);
    cudaMemcpy(A,dA,n*sizeof(int),cudaMemcpyDeviceToHost);
    print(A, n);
    //1 11 21 31 41 51 61 71 81 91 
    cudaFree(dA);
    free(A);
    return 0;
}

Overwriting increment.cu


In [17]:
%%script bash
nvcc increment.cu
./a.out

1 11 21 31 41 51 61 71 81 91 